In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt       # Standard plotting library


# Pretty display for notebooks
%matplotlib inline

# Load the Boston housing dataset
X = pd.read_csv('data/application_train_final.csv')
# Success - Display the first record
display(X.head(n=5))


,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,CNT_CHILDREN,CNT_FAM_MEMBERS,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,...,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,0.537494,0.489166,0.468924,0.245232,0.0,0.00,0.881337,0.035563,0.705433,0.739968,...,0,0,0,0,0,0,0,0,0,1
1,0.610092,0.746352,0.722714,0.279376,0.0,0.05,0.473087,0.066324,0.959566,0.810135,...,0,0,0,0,1,0,0,0,0,0
2,0.281783,0.244144,0.261437,0.114839,0.0,0.00,0.345593,0.012561,0.648326,0.813116,...,0,0,0,0,1,0,0,0,0,0
3,0.573739,0.430800,0.432648,0.197108,0.0,0.05,0.347884,0.169663,0.661387,0.858519,...,0,0,0,0,0,0,0,0,0,1
4,0.513463,0.540824,0.551329,0.184602,0.0,0.00,0.296071,0.169607,0.519522,0.746388,...,0,0,0,0,0,0,0,1,0,0


In [8]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

X.dropna(subset=['TARGET'], inplace=True)

target = 'TARGET'
IDcol = 'SK_ID_CURR'

/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [9]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : {}".format(metrics.accuracy_score(dtrain[target].values, dtrain_predictions)))
    print ("AUC Score (Train): {}".format(metrics.roc_auc_score(dtrain[target], dtrain_predprob)))
    print ("Precision (Train): {}".format(metrics.precision_score(dtrain[target].values, dtrain_predictions)))
    print ("Recall (Train): {}".format(metrics.recall_score(dtrain[target].values, dtrain_predictions)))
    tn, fp, fn, tp = confusion_matrix(dtrain[target].values, dtrain_predictions).ravel()
    print("TN: {}, FP: {}, FN: {}, TP: {}".format(tn, fp, fn, tp))
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [10]:
#Choose all predictors except target & IDcols
predictors = [x for x in X.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X, predictors)

KeyboardInterrupt: 

In [ ]:
param_test1 = {
 'max_depth':[3,10,2],
 'min_child_weight':[1,6,2]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X[predictors],X[target])
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [2]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X[predictors],X[target])
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

NameError: name 'GridSearchCV' is not defined

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=4,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2b.fit(X[predictors][:20000],X[target][:20000])

In [74]:
X2 = X
X2 = X2.drop[IDcol]
clf = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

81375 3366 6250 1175


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [36]:
clf = XGBClassifier(max_depth=6, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train, y_train)
predict = clf.predict(X_test)
tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

81375 3366 6250 1175


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
clf = XGBClassifier(max_depth=12, learning_rate=0.1,n_estimators=100,
                      n_jobs=16,scale_pos_weight=4,missing=np.nan,gamma=16,
                      eval_metric="auc",reg_lambda=40,reg_alpha=40)
clf = clf.fit(X_train[:20000], y_train[:20000])
predict = clf.predict(X_test[:20000])
tn, fp, fn, tp = confusion_matrix(y_test[:20000], predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

18067 326 1489 118


/Users/omar/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [68]:
clf = SVC()
clf = clf.fit(X_train[:20000], y_train[:20000])
predict = clf.predict(X_test[:20000])
tn, fp, fn, tp = confusion_matrix(y_test[:20000], predict).ravel()
print(tn, fp, fn, tp)

#scores = cross_val_score(model, train, target, cv=2, scoring='f1')

#print(scores)

18393 0 1607 0
